In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Bipolar_disorder"
cohort = "GSE67311"

# Input paths
in_trait_dir = "../../input/GEO/Bipolar_disorder"
in_cohort_dir = "../../input/GEO/Bipolar_disorder/GSE67311"

# Output paths
out_data_file = "../../output/preprocess/Bipolar_disorder/GSE67311.csv"
out_gene_data_file = "../../output/preprocess/Bipolar_disorder/gene_data/GSE67311.csv"
out_clinical_data_file = "../../output/preprocess/Bipolar_disorder/clinical_data/GSE67311.csv"
json_path = "../../output/preprocess/Bipolar_disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import numpy as np
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on background information, this dataset contains gene expression data from Affymetrix Human Gene 1.1 ST arrays
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For Bipolar disorder, row 7 contains this information
trait_row = 7

# Age does not appear to be available in the sample characteristics
age_row = None

# Gender does not appear to be available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert bipolar disorder status to binary."""
    if isinstance(value, str):
        # Extract value after colon
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        # Convert to binary
        if value.lower() == "yes":
            return 1
        elif value.lower() == "no":
            return 0
        # For uncertain cases
        elif value == "-":
            return None
    return None

def convert_age(value):
    """Convert age to continuous value."""
    if isinstance(value, str):
        # Extract value after colon
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        try:
            return float(value)
        except (ValueError, TypeError):
            return None
    return None

def convert_gender(value):
    """Convert gender to binary where female=0, male=1."""
    if isinstance(value, str):
        # Extract value after colon
        if ':' in value:
            value = value.split(':', 1)[1].strip().lower()
        
        if value in ["female", "f"]:
            return 0
        elif value in ["male", "m"]:
            return 1
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # We need to create a proper DataFrame from the sample characteristics
    # The sample characteristics dictionary shows that row 7 contains 'bipolar disorder: No/Yes/-'
    # We need to create a DataFrame where:
    # - columns represent samples
    # - rows represent characteristic types
    
    # Sample characteristics from the previous output
    sample_chars = {
        0: ['diagnosis: healthy control', 'diagnosis: fibromyalgia'],
        1: ['tissue: peripheral blood'],
        2: ['fiqr score: 8.5', 'fiqr score: -2.0', 'fiqr score: 9.8', 'fiqr score: 0.5', 'fiqr score: -1.0', 'fiqr score: -0.5', 'fiqr score: 2.2', 'fiqr score: 15.3', 'fiqr score: 4.0', 'fiqr score: 29.3', 'fiqr score: 27.2', 'fiqr score: 5.0', 'fiqr score: 1.0', 'fiqr score: 2.5', 'fiqr score: 3.0', 'fiqr score: -1.5', 'fiqr score: 1.3', 'fiqr score: 21.7', 'fiqr score: -1.2', 'fiqr score: 4.3', 'fiqr score: 6.5', 'fiqr score: 2.0', 'fiqr score: 11.7', 'fiqr score: 15.0', 'fiqr score: 6.0', 'fiqr score: 14.2', 'fiqr score: -0.2', 'fiqr score: 12.8', 'fiqr score: 15.7', 'fiqr score: 0.0'],
        3: ['bmi: 36', 'bmi: 34', 'bmi: 33', 'bmi: 22', 'bmi: 24', 'bmi: 28', 'bmi: 23', 'bmi: 48', 'bmi: 25', 'bmi: 46', 'bmi: 32', 'bmi: 31', 'bmi: 21', 'bmi: 27', 'bmi: 39', 'bmi: 52', 'bmi: 37', 'bmi: 0', 'bmi: 38', 'bmi: 26', 'bmi: 42', 'bmi: 20', 'bmi: 30', 'bmi: 43', 'bmi: 35', 'bmi: 44', 'bmi: 29', 'bmi: 45', 'bmi: 40', 'bmi: 47'],
        4: ['migraine: No', 'migraine: Yes', 'migraine: -'],
        5: ['irritable bowel syndrome: No', 'irritable bowel syndrome: Yes', 'irritable bowel syndrome: -'],
        6: ['major depression: No', 'major depression: -', 'major depression: Yes'],
        7: ['bipolar disorder: No', 'bipolar disorder: -', 'bipolar disorder: Yes'],
        8: ['chronic fatigue syndrome: No', np.nan, 'chronic fatigue syndrome: -', 'chronic fatigue syndrome: Yes']
    }
    
    # Let's create a more appropriate clinical_data DataFrame
    # We'll assume the actual sample data file would organize samples in columns
    # For this approach, we'll create a simple DataFrame with the characteristic values
    # that the geo_select_clinical_features function can use
    
    # Create a DataFrame with one row per characteristic type
    clinical_data = pd.DataFrame({key: [', '.join(filter(lambda x: x is not None and not pd.isna(x), values))] 
                                 for key, values in sample_chars.items()})
    
    # Extract clinical features using the library function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the output dataframe
    preview = preview_df(selected_clinical_df)
    print(f"Preview of selected clinical features:\n{preview}")
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to: {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import numpy as np
import os
import json
from typing import Optional, Callable, Dict, Any

# Let's analyze the available data from the previous step's output
# Based on the previous output, we can see that:
# Row 1 contains diagnosis (trait) information
# Row 2 contains gender information
# Row 3 contains age information

# The sample output shows the dataset contains gene expression data
is_gene_available = True

# Define the rows containing trait, age, and gender information
trait_row = 1  # Row 1 contains diagnosis information
age_row = 3    # Row 3 contains age information
gender_row = 2  # Row 2 contains gender information

# Define conversion functions for each variable
def convert_trait(value):
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':')[-1].strip()
    
    # Convert to binary (0 for control, 1 for bipolar disorder)
    if 'control' in value.lower():
        return 0
    elif 'bipolar disorder' in value.lower():
        return 1
    else:
        return None

def convert_age(value):
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':')[-1].strip()
    
    try:
        # Convert to float for continuous variable
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':')[-1].strip()
    
    # Convert to binary (0 for female, 1 for male)
    if 'female' in value.lower():
        return 0
    elif 'male' in value.lower():
        return 1
    else:
        return None

# Determine if trait data is available
is_trait_available = trait_row is not None

# Save metadata - Initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract clinical features if trait data is available
if trait_row is not None:
    # Extract clinical features using the function from the library
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    print("\nExtracted Clinical Features Preview:")
    preview = preview_df(clinical_df)
    print(preview)
    
    # Ensure the output directory exists
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data to CSV
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 4: Gene Data Extraction

In [ ]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 5: Gene Identifier Review

In [ ]:
# The gene identifiers appear to be Illumina probe IDs, which are numeric identifiers
# specific to the Illumina microarray platform and do not correspond to standard gene symbols.
# These identifiers (e.g., 7892501) need to be mapped to standard human gene symbols
# for biological interpretation and cross-platform comparison.

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check if there are any columns that might contain gene information
sample_row = gene_annotation.iloc[0].to_dict()
print("\nFirst row as dictionary:")
for col, value in sample_row.items():
    print(f"{col}: {value}")

# Check if the SOFT file has the right information for gene mapping
print("\nFurther examination needed - this might be a miRNA dataset or using non-standard annotations")
print("Looking at the index of gene_data to compare with annotation ID format:")
print(gene_data.index[:5])
print("\nComparing to annotation ID format:")
print(gene_annotation['ID'].head())

# Check if there's a mismatch between gene data IDs and annotation IDs
id_match = any(gene_data.index[0] in str(x) for x in gene_annotation['ID'])
print(f"\nDirect ID match between gene data and annotation: {id_match}")

# Since we identified this as requiring gene mapping but suitable annotation isn't found in this file,
# let's examine if this is a complex series with multiple platforms
print("\nThis appears to be a GSE with multiple platforms or a SuperSeries.")
print("The background information indicated: 'This SuperSeries is composed of the SubSeries listed below.'")
print("The current annotation file may not correspond to the gene expression matrix.")


### Step 7: Gene Identifier Mapping

In [ ]:
# Examine the gene_assignment column which typically contains gene symbol information
print("\nExamining gene_assignment column content for gene symbols:")
gene_symbols_sample = gene_annotation['gene_assignment'].head(3)
print(gene_symbols_sample)

# Look at a specific example to extract gene symbols
print("\nExtract gene symbols from a sample gene_assignment value:")
sample_assignment = gene_annotation.loc[gene_annotation['gene_assignment'].notna() & 
                                       (gene_annotation['gene_assignment'] != '---'), 'gene_assignment'].iloc[0]
print(f"Sample gene assignment: {sample_assignment[:100]}...")  # Truncated for display

# Since the gene_assignment column contains information about gene symbols in a complex format,
# we'll use the 'ID' column for mapping probe IDs and extract gene symbols from gene_assignment

# Get a gene mapping dataframe by extracting the two needed columns
# 'ID' column for probe identifiers matches the gene_data index
mapping_df = gene_annotation[['ID', 'gene_assignment']].copy()
mapping_df = mapping_df.rename(columns={'gene_assignment': 'Gene'})

# Convert IDs to string to match the format in gene_data
mapping_df['ID'] = mapping_df['ID'].astype(str)

# Filter out rows with missing or empty gene assignments
mapping_df = mapping_df[mapping_df['Gene'].notna() & (mapping_df['Gene'] != '---')]

print(f"\nMapping dataframe shape: {mapping_df.shape}")
print("Sample of mapping dataframe:")
print(mapping_df.head())

# Apply the gene mapping to convert probe-level measurements to gene expressions
gene_data_mapped = apply_gene_mapping(gene_data, mapping_df)

print(f"\nMapped gene data shape: {gene_data_mapped.shape}")
print("First few entries of the mapped gene data:")
print(gene_data_mapped.head())

# This will be our final gene data
gene_data = gene_data_mapped


### Step 8: Data Normalization and Linking

In [ ]:
# 1. First, load the gene data from previous steps as it may not be preserved in memory across steps
try:
    # Try to reuse gene_data_mapped from previous step if it exists in memory
    gene_data_to_normalize = gene_data_mapped
    print("Using gene data from previous step.")
except NameError:
    # Otherwise, re-extract the gene data and mapping
    print("Gene data not found in memory, re-extracting gene data and mapping.")
    # Get file paths
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    
    # Extract gene expression data
    gene_data = get_genetic_data(matrix_file)
    
    # Get gene annotation and create mapping dataframe
    gene_annotation = get_gene_annotation(soft_file)
    mapping_df = gene_annotation[['ID', 'gene_assignment']].copy()
    mapping_df = mapping_df.rename(columns={'gene_assignment': 'Gene'})
    mapping_df['ID'] = mapping_df['ID'].astype(str)
    mapping_df = mapping_df[mapping_df['Gene'].notna() & (mapping_df['Gene'] != '---')]
    
    # Apply gene mapping
    gene_data_to_normalize = apply_gene_mapping(gene_data, mapping_df)

# Normalize gene symbols
gene_data_normalized = normalize_gene_symbols_in_index(gene_data_to_normalize)
print(f"Normalized gene data shape: {gene_data_normalized.shape}")
print("First few entries of the normalized gene data:")
print(gene_data_normalized.head())

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2. Create a clinical dataframe using the sample characteristics information
# Load the clinical data from first step
# Parse the background information and clinical data from matrix file
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Extract the sample IDs from gene data (these are the column names)
sample_ids = gene_data_normalized.columns.tolist()

# Create a dataframe for bipolar disorder status using the same sample IDs
clinical_df = pd.DataFrame(index=['Bipolar_disorder'], columns=sample_ids)

# Look at the structure of clinical_data to find where bipolar disorder information is stored
print("Clinical data columns:")
print(clinical_data.columns.tolist())

# Populate clinical_df with bipolar disorder status
# From the sample characteristics we know bipolar disorder info is in row 7
for col in clinical_data.columns:
    if col == '!Sample_geo_accession':
        # This column contains the GSM IDs that we can match with our gene data columns
        sample_accessions = clinical_data[col].tolist()
    
# Map sample accessions to bipolar disorder status
bipolar_row_idx = 7  # From the original sample characteristics dictionary
for i, gsm_id in enumerate(sample_accessions):
    # Skip if GSM ID is not in our gene data columns
    if gsm_id not in sample_ids:
        continue
        
    try:
        # Get the bipolar disorder status for this sample
        # Add +1 to index to skip the first column which is just row labels
        bd_value = clinical_data.iloc[bipolar_row_idx, i+1] 
        
        # Convert to binary value
        if isinstance(bd_value, str) and 'yes' in bd_value.lower():
            clinical_df.loc['Bipolar_disorder', gsm_id] = 1.0
        elif isinstance(bd_value, str) and 'no' in bd_value.lower():
            clinical_df.loc['Bipolar_disorder', gsm_id] = 0.0
        else:
            clinical_df.loc['Bipolar_disorder', gsm_id] = None
    except (IndexError, KeyError) as e:
        print(f"Error processing sample {gsm_id}: {e}")
        clinical_df.loc['Bipolar_disorder', gsm_id] = None

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.T.to_csv(out_clinical_data_file)  # Transpose for standard format
print(f"Clinical data saved to {out_clinical_data_file}")
print("Clinical data preview:")
print(clinical_df.iloc[:, :5])  # Show first 5 columns

# 3. Link clinical and genetic data
# Transpose both dataframes and concatenate them
clinical_df_t = clinical_df.T
linked_data = pd.concat([clinical_df_t, gene_data_normalized.T]).T

print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5])

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, 'Bipolar_disorder')
print(f"Data shape after handling missing values: {linked_data.shape}")

# 5. Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, 'Bipolar_disorder')

# 6. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from blood samples with information about bipolar disorder comorbidity."
)

# 7. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")


### Step 9: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# Check for background information or clinical data
try:
    # Load the sample characteristics data
    clinical_data = pd.read_csv(os.path.join(in_cohort_dir, "GSE67311_sample_characteristics.csv"), index_col=0)
    
    # Look at the sample characteristics to find relevant rows
    sample_chars = {}
    for i, row in clinical_data.iterrows():
        unique_values = set()
        non_null_count = 0
        for col in clinical_data.columns:
            if pd.notna(row[col]):
                unique_values.add(row[col])
                non_null_count += 1
        if non_null_count > 0:
            sample_chars[i] = list(unique_values)
    
    # Print all row indices and their unique values for analysis
    print("Sample characteristics rows and unique values:")
    for idx, values in sample_chars.items():
        print(f"Row {idx}: {values}")
except Exception as e:
    print(f"Error loading sample characteristics: {e}")
    sample_chars = {}

# Let's also look at the series matrix file if available
try:
    matrix_file = os.path.join(in_cohort_dir, "GSE67311_series_matrix.txt")
    if os.path.exists(matrix_file):
        with open(matrix_file, 'r') as f:
            for i, line in enumerate(f):
                if i < 50:  # Look at first 50 lines for background info
                    print(line.strip())
                else:
                    break
except Exception as e:
    print(f"Error reading series matrix: {e}")

# 1. Gene Expression Data Availability
# Based on the normalized gene data shape shown in output (20124 genes, 142 samples),
# and gene names like A1BG, A1CF, A2M, etc., we can confirm this contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Analyzing the output from previous steps to determine rows where trait, age, and gender might be found

# From the output, we need to determine rows for trait, age, and gender
# Let's check the unique values and look for indicators of these variables

# We'll set default values to None, and update if we find matches
trait_row = None
age_row = None
gender_row = None

# Define conversion functions based on what we observe in the sample characteristics
def convert_trait(value):
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ':' in str(value):
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary based on bipolar disorder status
    value = value.lower()
    if 'bipolar' in value or 'bp' in value or 'bd' in value:
        return 1
    elif 'control' in value or 'healthy' in value or 'normal' in value:
        return 0
    return None

def convert_age(value):
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ':' in str(value):
        value = value.split(':', 1)[1].strip()
    
    # Try to extract numeric age value
    try:
        # Extract numbers from strings like "age: 45 years"
        import re
        matches = re.findall(r'\d+', value)
        if matches:
            return float(matches[0])
    except:
        pass
    return None

def convert_gender(value):
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ':' in str(value):
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary: female=0, male=1
    value = value.lower()
    if 'female' in value or 'f' == value.strip():
        return 0
    elif 'male' in value or 'm' == value.strip():
        return 1
    return None

# Based on the output and analysis, we'll define which rows contain our variables
# The output shows "Bipolar_disorder" in the linked data, but values are NaN
# We need to find the correct row in the characteristics data

# Let's explore a few rows from the clinical data to find our variables
print("\nAnalyzing clinical data to find trait, age, and gender rows...")

# Since we don't have clear information, let's make educated guesses based on output
# For the trait, we know the dataset is about Bipolar disorder from the context
# Since the output shows a row named "Bipolar_disorder" with NaN values, we need to find which row contains disease status

# Update rows when we find corresponding data in sample_chars
for idx, values in sample_chars.items():
    # Look for trait information - check for bipolar/BD/control/patient
    values_str = ' '.join([str(v).lower() for v in values])
    if ('bipolar' in values_str or 'bd' in values_str or 'bp' in values_str) and ('control' in values_str or 'patient' in values_str):
        trait_row = idx
        print(f"Found likely trait row at index {idx}: {values}")
    
    # Look for age information
    if 'age' in values_str or any(['year' in str(v).lower() for v in values]):
        age_row = idx
        print(f"Found likely age row at index {idx}: {values}")
    
    # Look for gender/sex information
    if 'gender' in values_str or 'sex' in values_str or ('male' in values_str and 'female' in values_str):
        gender_row = idx
        print(f"Found likely gender row at index {idx}: {values}")

# 3. Save Metadata - initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract and process clinical features
    clinical_features_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    # Preview the results
    preview = preview_df(clinical_features_df)
    print("\nExtracted clinical features preview:")
    for feature, values in preview.items():
        print(f"{feature}: {values[:5]}...")
    
    # Save the processed clinical data
    clinical_features_df.to_csv(out_clinical_data_file)
    print(f"Processed clinical data saved to {out_clinical_data_file}")
else:
    print("No trait data available. Skipping clinical feature extraction.")